## Predict

In [ ]:
from ultralytics import YOLO



Model = YOLO(
    model = './runs/detect/yolov8n_Infrared-Ocean-Target/weights/epoch100.pt' # Path to your yolo model
)

# Predict with model
PredictResult = Model.predict(
    source = './test.jpg' # Path to your test media
)

In [ ]:
import cv2
from ultralytics import YOLO


def Predicter(
    model: str,
    media: str
):
    # Load model
    Model = YOLO(model)

    def Predict(image):
        # Run YOLOv8 inference on the image
        results = Model(image)
        # Visualize the results on the frame
        annotated_frame = results[0].plot()
        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

    # Predict video
    if media.endswith('.mp4') or media.endswith('.avi'):
        # Open the video file
        Capture = cv2.VideoCapture(media)
        # Loop through the video frames
        while Capture.isOpened():
            # Read a frame from the video
            success, frame = Capture.read()
            if success:
                Predict(frame)
                # Break the loop if 'q' is pressed
                if cv2.waitKey(0) & 0xFF == ord("q"):
                    break
            else:
                # Break the loop if the end of the video is reached
                break
        # Release the video capture object and close the display window
        Capture.release()
        cv2.destroyAllWindows()

    # Predict image
    if media.endswith('.jpg') or media.endswith('.png'):
        # Open the image file
        image = cv2.imread(media)
        Predict(image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


Predicter(
    model = './runs/detect/yolov8n_Infrared-Ocean-Target/weights/epoch100.pt', # Path to your yolo model,
    media = './test.jpg' # Path to your test media
)